In [1]:
import pandas as pd 
import numpy as np 
# https://github.com/petrosDemetrakopoulos/RNN-Beatles-lyrics-generator
# https://github.com/starry91/Lyric-Generator#2-lyric-generator-based-on-word-level-rnn

In [2]:
data = pd.read_csv("sample-dataset.csv")
data.head()

,artist,genre,title,lyrics
0,Migos,rap,Stir Fry,"Woo, woo, woo, woo\nWoo, woo, woo, woo\n\nDanc..."
1,Snoop Dogg,rap,Drop It Like It‚Äôs Hot,"Snoop\nSnoop\n\nWhen the pimp's in the crib, m..."
2,Drake,rap,Headlines,I might be too strung out on compliments\nOver...
3,Lil Uzi Vert,rap,XO TOUR Llif3,"Are you alright?\nI'm alright, I'm quite alrig..."
4,Lil Uzi Vert,rap,The Way Life Goes,"That's true (That's true), that's right (That ..."


In [3]:
data['t-lyric'] = data['title'] + " @@@ " + data['lyrics']
data['t-lyric'] = data['t-lyric'].str.lower()
data.head()

,artist,genre,title,lyrics,t-lyric
0,Migos,rap,Stir Fry,"Woo, woo, woo, woo\nWoo, woo, woo, woo\n\nDanc...","stir fry @@@ woo, woo, woo, woo\nwoo, woo, woo..."
1,Snoop Dogg,rap,Drop It Like It‚Äôs Hot,"Snoop\nSnoop\n\nWhen the pimp's in the crib, m...",drop it like it‚äôs hot @@@ snoop\nsnoop\n\nwh...
2,Drake,rap,Headlines,I might be too strung out on compliments\nOver...,headlines @@@ i might be too strung out on com...
3,Lil Uzi Vert,rap,XO TOUR Llif3,"Are you alright?\nI'm alright, I'm quite alrig...",xo tour llif3 @@@ are you alright?\ni'm alrigh...
4,Lil Uzi Vert,rap,The Way Life Goes,"That's true (That's true), that's right (That ...",the way life goes @@@ that's true (that's true...


In [4]:
stopChars = [',','(',')','.','-','[',']','"']
# preprocessing the corpus by converting all letters to lowercase, 
# replacing blank lines with blank string and removing special characters
def preprocessText(text):
#     text = text.replace('\n', ' ').replace('\t','')
    processedText = text.lower()
    for char in stopChars:
        processedText = processedText.replace(char,'')
    return processedText
data['t-lyric'] = data['t-lyric'].apply(preprocessText)

In [5]:
# tokenization 
def corpusToList(corpus):
    corpusList = [w for w in corpus.split(' ')] 
    corpusList = [i for i in corpusList if i] #removing empty strings from list
    return corpusList
data['t-lyric'] = data['t-lyric'].apply(corpusToList)

In [6]:
data.head()

,artist,genre,title,lyrics,t-lyric
0,Migos,rap,Stir Fry,"Woo, woo, woo, woo\nWoo, woo, woo, woo\n\nDanc...","[stir, fry, @@@, woo, woo, woo, woo\nwoo, woo,..."
1,Snoop Dogg,rap,Drop It Like It‚Äôs Hot,"Snoop\nSnoop\n\nWhen the pimp's in the crib, m...","[drop, it, like, it‚äôs, hot, @@@, snoop\nsnoo..."
2,Drake,rap,Headlines,I might be too strung out on compliments\nOver...,"[headlines, @@@, i, might, be, too, strung, ou..."
3,Lil Uzi Vert,rap,XO TOUR Llif3,"Are you alright?\nI'm alright, I'm quite alrig...","[xo, tour, llif3, @@@, are, you, alright?\ni'm..."
4,Lil Uzi Vert,rap,The Way Life Goes,"That's true (That's true), that's right (That ...","[the, way, life, goes, @@@, that's, true, that..."


In [7]:
# trim each word for leading or trailing spaces / tabs.
map(str.strip, data['t-lyric']) # trim words

In [8]:
data.head()

,artist,genre,title,lyrics,t-lyric
0,Migos,rap,Stir Fry,"Woo, woo, woo, woo\nWoo, woo, woo, woo\n\nDanc...","[stir, fry, @@@, woo, woo, woo, woo\nwoo, woo,..."
1,Snoop Dogg,rap,Drop It Like It‚Äôs Hot,"Snoop\nSnoop\n\nWhen the pimp's in the crib, m...","[drop, it, like, it‚äôs, hot, @@@, snoop\nsnoo..."
2,Drake,rap,Headlines,I might be too strung out on compliments\nOver...,"[headlines, @@@, i, might, be, too, strung, ou..."
3,Lil Uzi Vert,rap,XO TOUR Llif3,"Are you alright?\nI'm alright, I'm quite alrig...","[xo, tour, llif3, @@@, are, you, alright?\ni'm..."
4,Lil Uzi Vert,rap,The Way Life Goes,"That's true (That's true), that's right (That ...","[the, way, life, goes, @@@, that's, true, that..."


In [9]:
corpus_words = [x for sublist in data['t-lyric'] for x in sublist]
vocab = sorted(set(corpus_words))
print('vocab length:', len(corpus_words))
print('Unique words in corpus: {}'.format(len(vocab)))

vocab length: 2810
Unique words in corpus: 797


In [10]:
# creating numeric map; representing words with numberes 
# map specific number to each specific word of our corpus, and vice versa 
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpus_words])

## Prediction

- User inputs a song title, and how many words they want the song to be. 
- Network does, for example, 100 predictions, and in the training phrase we know what word we need to generate. 
- (genre, song title); have a marker that it's the end of the title 

## Character Level LSTM

Attempting a character level-RNN for a single genre
https://www.kaggle.com/super13579/let-s-auto-write-the-deep-purple-lysics-pytorch

In [11]:
data1 = pd.read_csv("sample-dataset.csv")
data1['lyrics']= data1['lyrics'].apply(preprocessText)
print(data1.head())

         artist genre                    title  \
0         Migos   rap                 Stir Fry   
1    Snoop Dogg   rap  Drop It Like It‚Äôs Hot   
2         Drake   rap                Headlines   
3  Lil Uzi Vert   rap            XO TOUR Llif3   
4  Lil Uzi Vert   rap        The Way Life Goes   

                                              lyrics  
0  woo woo woo woo\nwoo woo woo woo\n\ndance with...  
1  snoop\nsnoop\n\nwhen the pimp's in the crib ma...  
2  i might be too strung out on compliments\nover...  
3  are you alright?\ni'm alright i'm quite alrigh...  
4  that's true that's true that's right that righ...  


In [12]:
DP_text = data1['lyrics'].str.cat(sep='\n').lower()
print('corpus length:', len(DP_text))

corpus length: 15232


In [13]:
# Counting characters appeared in all lyrics
chars = sorted(list(set(DP_text)))
print(chars)
print('total chars:', len(chars))

['\n', ' ', '!', "'", '4', '7', '8', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '´', '¶', 'ä', 'î', 'ò', '‚', '√']
total chars: 41


In [14]:
# Create a dictionary of characters, see the index of characters.
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, "'": 3, '4': 4, '7': 5, '8': 6, '?': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'j': 17, 'k': 18, 'l': 19, 'm': 20, 'n': 21, 'o': 22, 'p': 23, 'q': 24, 'r': 25, 's': 26, 't': 27, 'u': 28, 'v': 29, 'w': 30, 'x': 31, 'y': 32, 'z': 33, '´': 34, '¶': 35, 'ä': 36, 'î': 37, 'ò': 38, '‚': 39, '√': 40}


In [15]:
seq_length = 50 # The sentence window size
step = 1 # The steps between the windows
sentences = []
next_chars = []

# Create Target and sentences window
for i in range(0, len(DP_text) - seq_length, step):
    # range from current index to sequence length charaters
    sentences.append(DP_text[i: i + seq_length])  
    next_chars.append(DP_text[i + seq_length]) # the next character
    
sentences = np.array(sentences)
next_chars = np.array(next_chars)

#Print Sentence Window and next charaters
print('Sentence Window')
print (sentences[:5])
print('Target characters')
print (next_chars[:5])
print('Number of sequences:', len(sentences))

Sentence Window
['woo woo woo woo\nwoo woo woo woo\n\ndance with my dog'
 'oo woo woo woo\nwoo woo woo woo\n\ndance with my dogs'
 'o woo woo woo\nwoo woo woo woo\n\ndance with my dogs '
 ' woo woo woo\nwoo woo woo woo\n\ndance with my dogs i'
 'woo woo woo\nwoo woo woo woo\n\ndance with my dogs in']
Target characters
['s' ' ' 'i' 'n' ' ']
Number of sequences: 15182


In [ ]:
# transferring the character to index 
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences),seq_length))
    y = np.zeros((len(sentences)))
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t] = char_to_int[char]
        y[i] = char_to_int[next_chars[i]]
    return X, y

In [ ]:
train_x,train_y = getdata(sentences, next_chars)
print(train_x)
print('Shape of training_x:', train_x.shape)
print('Shape of training_y:', train_y.shape)

## Building out the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Simple_LSTM(nn.Module):
    def __init__(self,n_vocab,hidden_dim, embedding_dim,dropout = 0.2):
        super(Simple_LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,dropout = dropout,num_layers = 2)
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)
        self.fc = nn.Linear(hidden_dim, n_vocab)
    
    def forward(self, seq_in):
        # for LSTM, input should be (Sequnce_length,batchsize,hidden_layer), so we need to transpose the input
        embedded = self.embeddings(seq_in.t()) 
        lstm_out, _ = self.lstm(embedded)
        # Only need to keep the last character 
        ht=lstm_out[-1] 
        out = self.fc(ht)
        return out

In [ ]:
X_train_tensor = torch.tensor(train_x, dtype=torch.long)
Y_train_tensor = torch.tensor(train_y, dtype=torch.long)

In [ ]:
from torch.utils.data import Dataset, DataLoader
train = torch.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)

In [ ]:
model = Simple_LSTM(47,256,256)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002) # Using Adam optimizer

In [ ]:
import time # Add time counter
avg_losses_f = []
n_epochs=10

for epoch in range(n_epochs):
    print("Epoch: ", epoch)
    start_time = time.time()
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    avg_loss = 0.
    for i, (x_batch, y_batch) in enumerate(train_loader):
        y_pred = model(x_batch)
        
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        avg_loss+= loss.item()/len(train_loader)
        
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, elapsed_time))
    
    avg_losses_f.append(avg_loss)    
    
print('All \t loss={:.4f} \t '.format(np.average(avg_losses_f)))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(avg_losses_f)
plt.xlabel('Epoch')
plt.ylabel('Loss value')
plt.show()

creating a function that can sample an index from a probability array 


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

validate the model

In [ ]:
# Define the start sentence
# sentence = 'i read in the news\nthat the average man\nplease kis'
sentence = 'i put the new forgis on the g\ni trap until the blo'
variance = 0.25
generated = ''
original = sentence
window = sentence

for i in range(400):
    x = np.zeros((1, seq_length))
    for t, char in enumerate(window):
        x[0, t] = char_to_int[char] # Change the sentence to index vector shape (1,50)
        
    x_in = Variable(torch.LongTensor(x))
    pred = model(x_in)
    pred = np.array(F.softmax(pred, dim=1).data[0].cpu())
    next_index = sample(pred, variance)
    next_char = int_to_char[next_index] # index to char

    generated += next_char
    window = window[1:] + next_char # Update Window for next char predict
    
print(original + generated)

# Word Level LSTM

In [16]:
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
data1 = pd.read_csv("sample-dataset.csv")
data1['lyrics']= data1['lyrics'].apply(preprocessText)
print(data1.head())

         artist genre                    title  \
0         Migos   rap                 Stir Fry   
1    Snoop Dogg   rap  Drop It Like It‚Äôs Hot   
2         Drake   rap                Headlines   
3  Lil Uzi Vert   rap            XO TOUR Llif3   
4  Lil Uzi Vert   rap        The Way Life Goes   

                                              lyrics  
0  woo woo woo woo\nwoo woo woo woo\n\ndance with...  
1  snoop\nsnoop\n\nwhen the pimp's in the crib ma...  
2  i might be too strung out on compliments\nover...  
3  are you alright?\ni'm alright i'm quite alrigh...  
4  that's true that's true that's right that righ...  


In [20]:
DP_text = word_tokenize(data1['lyrics'].str.cat(sep='\n').lower())
print('corpus length:', len(DP_text))
#todo(dlee, tokenize words)

corpus length: 3440


In [24]:
# Counting characters appeared in all lyrics
words = sorted(list(set(DP_text)))
print('total words:', len(words))

total words: 650


In [25]:
# Create a dictionary of characters, see the index of characters.
char_to_int = dict((c, i) for i, c in enumerate(words))
int_to_char = dict((i, c) for i, c in enumerate(words))

In [26]:
seq_length = 10 # The sentence window size
step = 1 # The steps between the windows
sentences = []
next_chars = []

# Create Target and sentences window
for i in range(0, len(DP_text) - seq_length, step):
    # range from current index to sequence length charaters
    sentences.append(DP_text[i: i + seq_length])  
    next_chars.append(DP_text[i + seq_length]) # the next character
    
sentences = np.array(sentences)
next_chars = np.array(next_chars)

#Print Sentence Window and next charaters
print('Sentence Window')
print (sentences[:5])
print('Target characters')
print (next_chars[:5])
print('Number of sequences:', len(sentences))

Sentence Window
[['woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'dance' 'with']
 ['woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'dance' 'with' 'my']
 ['woo' 'woo' 'woo' 'woo' 'woo' 'woo' 'dance' 'with' 'my' 'dogs']
 ['woo' 'woo' 'woo' 'woo' 'woo' 'dance' 'with' 'my' 'dogs' 'in']
 ['woo' 'woo' 'woo' 'woo' 'dance' 'with' 'my' 'dogs' 'in' 'the']]
Target characters
['my' 'dogs' 'in' 'the' 'nighttime']
Number of sequences: 3430


In [27]:
# transferring the character to index 
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences),seq_length))
    y = np.zeros((len(sentences)))
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t] = char_to_int[char]
        y[i] = char_to_int[next_chars[i]]
    return X, y

In [28]:
train_x,train_y = getdata(sentences, next_chars)
print(train_x)
print('Shape of training_x:', train_x.shape)
print('Shape of training_y:', train_y.shape)

[[632. 632. 632. ... 632. 145. 629.]
 [632. 632. 632. ... 145. 629. 364.]
 [632. 632. 632. ... 629. 364. 162.]
 ...
 [ 86. 284. 320. ... 284. 628. 284.]
 [284. 320. 547. ... 628. 284. 374.]
 [320. 547. 236. ... 284. 374. 348.]]
Shape of training_x: (3430, 10)
Shape of training_y: (3430,)


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Simple_LSTM(nn.Module):
    def __init__(self,n_vocab,hidden_dim, embedding_dim,dropout = 0.2):
        super(Simple_LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,dropout = dropout,num_layers = 2)
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)
        self.fc = nn.Linear(hidden_dim, n_vocab)
    
    def forward(self, seq_in):
        # for LSTM, input should be (Sequnce_length,batchsize,hidden_layer), so we need to transpose the input
        embedded = self.embeddings(seq_in.t()) 
        lstm_out, _ = self.lstm(embedded)
        # Only need to keep the last character 
        ht=lstm_out[-1] 
        out = self.fc(ht)
        return out

In [30]:
X_train_tensor = torch.tensor(train_x, dtype=torch.long)
Y_train_tensor = torch.tensor(train_y, dtype=torch.long)
print('Shape of training_x:', X_train_tensor.shape)
print('Shape of training_y:', Y_train_tensor.shape)

Shape of training_x: torch.Size([3430, 10])
Shape of training_y: torch.Size([3430])


In [31]:
from torch.utils.data import Dataset, DataLoader
train = torch.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)

In [32]:
model = Simple_LSTM(len(words),256,256)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002) # Using Adam optimizer

In [33]:
import time # Add time counter
avg_losses_f = []
n_epochs=20

for epoch in range(n_epochs):
    print("Epoch: ", epoch)
    start_time = time.time()
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    avg_loss = 0.
    for i, (x_batch, y_batch) in enumerate(train_loader):
        y_pred = model(x_batch)
        
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        avg_loss+= loss.item()/len(train_loader)
        
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, elapsed_time))
    
    avg_losses_f.append(avg_loss)    
    
print('All \t loss={:.4f} \t '.format(np.average(avg_losses_f)))

Epoch:  0
Epoch 1/20 	 loss=5.8391 	 time=3.12s
Epoch:  1
Epoch 2/20 	 loss=5.3785 	 time=2.98s
Epoch:  2
Epoch 3/20 	 loss=4.9413 	 time=2.92s
Epoch:  3
Epoch 4/20 	 loss=4.4266 	 time=3.15s
Epoch:  4
Epoch 5/20 	 loss=3.8698 	 time=3.20s
Epoch:  5
Epoch 6/20 	 loss=3.3261 	 time=3.07s
Epoch:  6
Epoch 7/20 	 loss=2.8072 	 time=3.13s
Epoch:  7
Epoch 8/20 	 loss=2.3601 	 time=3.17s
Epoch:  8
Epoch 9/20 	 loss=1.9756 	 time=3.02s
Epoch:  9
Epoch 10/20 	 loss=1.5993 	 time=3.17s
Epoch:  10
Epoch 11/20 	 loss=1.2876 	 time=3.14s
Epoch:  11
Epoch 12/20 	 loss=0.9869 	 time=3.14s
Epoch:  12
Epoch 13/20 	 loss=0.7217 	 time=3.20s
Epoch:  13
Epoch 14/20 	 loss=0.5157 	 time=3.15s
Epoch:  14
Epoch 15/20 	 loss=0.3938 	 time=3.20s
Epoch:  15
Epoch 16/20 	 loss=0.2901 	 time=3.13s
Epoch:  16
Epoch 17/20 	 loss=0.2170 	 time=3.17s
Epoch:  17
Epoch 18/20 	 loss=0.1737 	 time=3.22s
Epoch:  18
Epoch 19/20 	 loss=0.1460 	 time=3.23s
Epoch:  19
Epoch 20/20 	 loss=0.1319 	 time=3.21s
All 	 loss=2.0694 	

In [34]:
import matplotlib.pyplot as plt

plt.plot(avg_losses_f)
plt.xlabel('Epoch')
plt.ylabel('Loss value')
plt.show()

<Figure size 640x480 with 1 Axes>

In [35]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [42]:
# Define the start sentence
# sentence = 'i read in the news\nthat the average man\nplease kis'
sentence = ["she", "got", "forever", "addicted", "to", "a", "body", "killer", "spillin", "ice"]
variance = 1
generated = []
original = sentence
window = sentence

for i in range(400):
    x = np.zeros((1, seq_length))
    for t, char in enumerate(window):
        x[0, t] = char_to_int[char] # Change the sentence to index vector shape (1,50)
        
    x_in = Variable(torch.LongTensor(x))
    pred = model(x_in)
    pred = np.array(F.softmax(pred, dim=1).data[0].cpu())
    next_index = sample(pred, variance)
    next_char = int_to_char[next_index] # index to char

    generated = generated + [next_char]
    window = window[1:] + [next_char] # Update Window for next char predict
    
print(" ".join(original + generated))

she got forever addicted to a body killer spillin ice with the chickens like suicide wrist red take it i take a second ding cars real you big should four take your rick diamonds on the million hey to give you fuck faded them own look and i got a niggas premiere boss to the left pockets fallin and i roll the best weed ‚äòcause i got it goin ' on i 'm a gangsta but y'all y'all knew that leather big boss yeah i had to do that i keep a blue flag hangin ' all my money but in the way that she treat me gon ' leave you wo n't leave me i call it that casanova she say i 'm insane yeah i might blow my brain out hey xanny help the pain yeah please xanny make it go away i 'm committed not addicted but it keep control of me all the pain now i ca n't feel it i swear it 's listening she to the plan other all way my bands to with the money all my friends are dead push me to the edge all my friends are dead push me to the edge that 's all red inside all white like somethin ' you ride a sled down i just 